In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import jupyter

In [2]:
# Importing the original understat dataframes and sorting them by date

# GitHub Actions use only:

eng_df = pd.read_excel(r'EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet',index_col=[0])
eng_df = eng_df.sort_values(by='date',ascending=False)
fra_df = pd.read_excel(r'EDA_DF/Ligue1_understat.xlsx',sheet_name='gamesheet',index_col=[0])
fra_df = fra_df.sort_values(by='date',ascending=False)
ger_df = pd.read_excel(r'EDA_DF/Bundesliga_understat.xlsx',sheet_name='gamesheet',index_col=[0])
ger_df = ger_df.sort_values(by='date',ascending=False)
ita_df = pd.read_excel(r'EDA_DF/SerieA_understat.xlsx',sheet_name='gamesheet',index_col=[0])
ita_df = ita_df.sort_values(by='date',ascending=False)
spa_df = pd.read_excel(r'EDA_DF/LaLiga_understat.xlsx',sheet_name='gamesheet',index_col=[0])
spa_df = spa_df.sort_values(by='date',ascending=False)
fra_df


FileNotFoundError: [Errno 2] No such file or directory: 'EDA_DF/EPL_understat.xlsx'

In [3]:
# Importing the original understat dataframes and sorting them by date

# VSCode local use only:

# eng_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet',index_col=[0])
# eng_df = eng_df.sort_values(by='date',ascending=False)
# fra_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='gamesheet',index_col=[0])
# fra_df = fra_df.sort_values(by='date',ascending=False)
# ger_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='gamesheet',index_col=[0])
# ger_df = ger_df.sort_values(by='date',ascending=False)
# ita_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='gamesheet',index_col=[0])
# ita_df = ita_df.sort_values(by='date',ascending=False)
# spa_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='gamesheet',index_col=[0])
# spa_df = spa_df.sort_values(by='date',ascending=False)
# fra_df

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,QS,QSC,Pts,Opp,BTTS,Blank,CS,first_goal,first_conc
0,2024-04-21 19:00:00,Lyon23632,Lyon,a,0.667,3.178,1,4,17,14,0,5,0,Paris Saint Germain,1,0,0,0,1
0,2024-04-21 19:00:00,Paris Saint Germain23632,Paris Saint Germain,h,3.178,0.667,4,1,14,17,5,0,3,Lyon,1,0,0,1,0
0,2024-04-21 17:00:00,Toulouse23633,Toulouse,h,0.655,1.095,2,2,10,11,1,1,1,Marseille,1,0,0,0,1
0,2024-04-21 17:00:00,Marseille23633,Marseille,a,1.095,0.655,2,2,11,10,1,1,1,Toulouse,1,0,0,1,0
0,2024-04-21 15:05:00,Brest23640,Brest,h,0.641,1.313,0,2,12,8,0,1,0,Monaco,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-03-12 19:45:00,Strasbourg19916,Strasbourg,a,2.232,1.718,2,2,24,7,2,2,1,Marseille,1,0,0,0,1
0,2023-03-11 20:00:00,Brest19917,Brest,h,0.751,2.674,1,2,7,17,1,4,0,Paris Saint Germain,1,0,0,0,1
0,2023-03-11 20:00:00,Paris Saint Germain19917,Paris Saint Germain,a,2.674,0.751,2,1,17,7,4,1,3,Brest,1,0,0,1,0
0,2023-03-10 20:00:00,Lille19915,Lille,h,2.353,1.058,3,3,8,6,4,2,1,Lyon,1,0,0,1,0


In [4]:
# Home/Away Tables rolling season (Historic)

def home_away_table(df):
    # changes to df
    df['match_ID'] = df['ID'].apply(lambda x:x[-5:])
    df['match_ID'] = pd.to_numeric(df['match_ID'])
    df['played'] = 1
    df['0_0'] = df.apply(lambda x: 1 if x['CS'] == 1 and x['Blank'] ==1 else 0, axis = 1)
    
    # new df
    df_h_a = df.groupby(["Team","H/A"]).agg({'played':'sum', 'xG':'sum', 'xGC':'sum','G':'sum', 'GC':'sum', 'Pts':'sum',
                                             'S':'sum', 'SC':'sum', 'QS': 'sum','QSC':'sum',
                                             'BTTS':'sum', 'Blank':'sum', 'CS':'sum','first_goal':'sum','first_conc':'sum','0_0':'sum'})
    h_a_sep = [x for _, x in df_h_a.groupby("H/A")]
    away_table = h_a_sep[0].droplevel("H/A")
    home_table = h_a_sep[1].droplevel("H/A")
    home_table = home_table.apply(pd.to_numeric)
    away_table = away_table.apply(pd.to_numeric)
    home_table['GD'] = home_table['G'] - home_table['GC']
    away_table['GD'] = away_table['G'] - away_table['GC']
    home_table['sample'] = home_table['played']**2
    away_table['sample'] = home_table['played']**2
    home_table = home_table[~(home_table['played'] <= 10)]
    away_table = away_table[~(away_table['played'] <= 10)]
    home_table = home_table.iloc[:,1:].div(home_table.played, axis=0)
    away_table = away_table.iloc[:,1:].div(away_table.played, axis=0)
    home_table['btts_if_g'] = home_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    away_table['btts_if_g'] = away_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    home_table['btts_if_c'] = home_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)
    away_table['btts_if_c'] = away_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1) 
    q1_h = home_table['Pts'].quantile(0.25)
    q1_a = away_table['Pts'].quantile(0.25)
    q3_h = home_table['Pts'].quantile(0.75)
    q3_a = away_table['Pts'].quantile(0.75)
    home_table['Rank'] = home_table.apply(lambda x: 3 if x['Pts'] <= q1_h else( 1 if x['Pts'] >= q3_h else 2), axis=1) 
    away_table['Rank'] = away_table.apply(lambda x: 3 if x['Pts'] <= q1_a else( 1 if x['Pts'] >= q3_a else 2), axis=1) 
    home_table = home_table.sort_values('Pts', ascending=False)
    away_table = away_table.sort_values('Pts', ascending=False)

    return home_table, away_table

eng_table_h, eng_table_a = home_away_table(eng_df)
fra_table_h, fra_table_a = home_away_table(fra_df)
ger_table_h, ger_table_a = home_away_table(ger_df)
ita_table_h, ita_table_a = home_away_table(ita_df)
spa_table_h, spa_table_a = home_away_table(spa_df)
eng_table_h

,xG,xGC,G,GC,Pts,S,SC,QS,QSC,BTTS,Blank,CS,first_goal,first_conc,0_0,GD,sample,btts_if_g,btts_if_c,Rank
Team,,,,,,,,,,,,,,,,,,,,
Manchester City,2.235850,0.798700,2.500000,0.800000,2.500000,18.400000,6.650000,2.600000,1.100000,0.650000,0.050000,0.350000,0.800000,0.150000,0.050000,1.700000,20.0,0.625000,1.000000,1
Liverpool,2.854500,1.007700,2.450000,0.950000,2.450000,22.750000,8.850000,3.600000,1.600000,0.650000,0.100000,0.300000,0.600000,0.350000,0.050000,1.500000,20.0,0.583333,0.857143,1
Aston Villa,2.234737,1.461737,2.473684,1.315789,2.315789,14.368421,9.736842,2.789474,2.210526,0.736842,0.052632,0.210526,0.684211,0.315789,0.000000,1.157895,19.0,0.692308,0.833333,1
Arsenal,2.322842,0.840211,2.421053,1.000000,2.315789,17.210526,7.789474,3.000000,1.157895,0.526316,0.157895,0.315789,0.736842,0.263158,0.000000,1.421053,19.0,0.571429,0.400000,1
Tottenham,2.133000,1.535389,2.000000,1.333333,2.166667,17.111111,12.166667,2.777778,2.055556,0.833333,0.000000,0.166667,0.666667,0.333333,0.000000,0.666667,18.0,0.750000,1.000000,1
Newcastle United,2.793850,1.205250,2.300000,1.150000,2.000000,17.450000,10.900000,3.900000,1.250000,0.500000,0.100000,0.450000,0.750000,0.200000,0.050000,1.150000,20.0,0.466667,0.750000,2
Manchester United,1.862444,1.697278,1.555556,1.277778,1.944444,16.555556,14.055556,2.222222,1.888889,0.500000,0.166667,0.333333,0.444444,0.555556,0.000000,0.277778,18.0,0.250000,0.700000,2
Brighton,2.112778,1.495611,2.111111,1.388889,1.833333,18.111111,10.277778,2.555556,1.777778,0.777778,0.111111,0.166667,0.555556,0.388889,0.055556,0.722222,18.0,0.800000,0.857143,2
Chelsea,2.472444,1.414889,2.111111,1.555556,1.666667,16.277778,12.722222,3.388889,1.722222,0.611111,0.166667,0.222222,0.611111,0.388889,0.000000,0.555556,18.0,0.636364,0.571429,2


In [5]:
# Apply rank to overall dataframe, and then group in tables by rank like in home vs away

def rank_to_df(rank_df_h,rank_df_a,df):
    rank_df_h['Team index'] = rank_df_h.index
    rank_df_a['Team index'] = rank_df_a.index

    rank1_h = rank_df_h.loc[rank_df_h['Rank'] == 1, ['Team index']].values.T.tolist()
    rank2_h = rank_df_h.loc[rank_df_h['Rank'] == 2, ['Team index']].values.T.tolist()
    rank3_h = rank_df_h.loc[rank_df_h['Rank'] == 3, ['Team index']].values.T.tolist()
    rank1_a = rank_df_a.loc[rank_df_a['Rank'] == 1, ['Team index']].values.T.tolist()
    rank2_a = rank_df_a.loc[rank_df_a['Rank'] == 2, ['Team index']].values.T.tolist()
    rank3_a = rank_df_a.loc[rank_df_a['Rank'] == 3, ['Team index']].values.T.tolist()

    df['Rankvs'] = df.apply(lambda x: 1 if x['H/A'] =="a" and x['Opp'] in rank1_h[0] else(1 if x['H/A'] == "h" and x['Opp'] in rank1_a[0] else
                                                                                     (2 if x['H/A'] == "a" and x['Opp'] in rank2_h[0] else
                                                                                      (2 if x['H/A'] == "h" and x['Opp'] in rank2_a[0] else
                                                                                       (3 if x['H/A'] == "a" and x['Opp'] in rank3_h[0] else 3
                                                                                       )))), axis=1) 
    
    df['Rank'] = df.apply(lambda x: 1 if x['H/A'] =="h" and x['Team'] in rank1_h[0] else(1 if x['H/A'] == "a" and x['Team'] in rank1_a[0] else
                                                                                     (2 if x['H/A'] == "h" and x['Team'] in rank2_h[0] else
                                                                                      (2 if x['H/A'] == "a" and x['Team'] in rank2_a[0] else
                                                                                       (3 if x['H/A'] == "h" and x['Team'] in rank3_h[0] else 3
                                                                                       )))), axis=1) 
    df['win'] = df.apply(lambda x: 1 if x['Pts'] == 3 else 0, axis=1)
    df['draw'] = df.apply(lambda x: 1 if x['Pts'] == 1 else 0, axis=1)
    df['lose'] = df.apply(lambda x: 1 if x['Pts'] == 0 else 0, axis=1)

    
    return df

eng_df_updated = rank_to_df(eng_table_h,eng_table_a,eng_df)
fra_df_updated = rank_to_df(fra_table_h,fra_table_a,fra_df)
ger_df_updated = rank_to_df(ger_table_h,ger_table_a,ger_df)
ita_df_updated = rank_to_df(ita_table_h,ita_table_a,ita_df)
spa_df_updated = rank_to_df(spa_table_h,spa_table_a,spa_df)
eng_df_updated

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,...,first_goal,first_conc,match_ID,played,0_0,Rankvs,Rank,win,draw,lose
0,2024-04-21 15:30:00,Fulham22229,Fulham,h,0.851,1.024,1,3,12,14,...,0,1,22229,1,0,1,2,0,0,1
0,2024-04-21 15:30:00,Liverpool22229,Liverpool,a,1.024,0.851,3,1,14,12,...,1,0,22229,1,0,2,1,1,0,0
0,2024-04-21 14:00:00,West Ham22227,West Ham,a,1.345,3.863,2,5,5,19,...,0,1,22227,1,0,2,2,0,0,1
0,2024-04-21 14:00:00,Bournemouth22225,Bournemouth,a,2.127,2.940,1,3,14,14,...,1,0,22225,1,0,1,2,0,0,1
0,2024-04-21 14:00:00,Aston Villa22225,Aston Villa,h,2.940,2.127,3,1,14,14,...,0,1,22225,1,0,2,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-04-29 14:00:00,Nottingham Forest18534,Nottingham Forest,a,0.965,1.375,1,2,5,14,...,1,0,18534,1,0,2,3,0,0,1
0,2023-04-29 11:30:00,Crystal Palace18536,Crystal Palace,h,2.319,2.059,4,3,16,8,...,0,1,18536,1,0,2,2,1,0,0
0,2023-04-29 11:30:00,West Ham18536,West Ham,a,2.059,2.319,3,4,8,16,...,1,0,18536,1,0,2,2,0,0,1
0,2023-04-26 19:00:00,Manchester City18531,Manchester City,h,2.012,0.398,4,1,14,8,...,1,0,18531,1,0,1,1,1,0,0


In [6]:
# Form Dataframe (Last 10 games)

def last10(df):
    temp_df = df
    temp_df['played'] = 1
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['date_int'] = (datetime.datetime.now()) - (temp_df['date'])
    temp_df['0_0'] = temp_df.apply(lambda x: 1 if x['CS'] == 1 and x['Blank'] == 1 else 0, axis = 1)
    temp_df['btts_if_g'] = temp_df.apply(lambda x: 1 if x['first_goal'] == 1 and x['BTTS'] == 1 else 0, axis = 1)
    temp_df['btts_if_c'] = temp_df.apply(lambda x: 1 if x['first_conc'] == 1 and x['BTTS'] == 1 else 0, axis = 1)
    temp_df['win'] = temp_df.apply(lambda x: 1 if x['Pts'] == 3 else 0, axis=1)
    temp_df['draw'] = temp_df.apply(lambda x: 1 if x['Pts'] == 1 else 0, axis=1)
    temp_df['lose'] = temp_df.apply(lambda x: 1 if x['Pts'] == 0 else 0, axis=1)

    temp_df = temp_df.groupby(['Team'],as_index=False).apply(lambda x: x.nsmallest(10, 'date_int'),include_groups=True)
    temp_df.reset_index(inplace=True)
    temp_df.drop(['level_0','level_1','date_int'],axis=1,inplace=True)
    return temp_df

eng_l10 = last10(eng_df_updated)
fra_l10 = last10(fra_df_updated)
ger_l10 = last10(ger_df_updated)
ita_l10 = last10(ita_df_updated)
spa_l10 = last10(spa_df_updated)
eng_l10

C:\Users\User\AppData\Local\Temp\ipykernel_12388\3052241984.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = temp_df.groupby(['Team'],as_index=False).apply(lambda x: x.nsmallest(10, 'date_int'),include_groups=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12388\3052241984.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = temp_df.groupby(['Team'],as_index=False).apply(lambda x: x

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,...,match_ID,played,0_0,Rankvs,Rank,win,draw,lose,btts_if_g,btts_if_c
0,2024-04-20 18:30:00,Arsenal22234,Arsenal,a,1.662,0.165,2,0,24,5,...,22234,1,0,2,1,1,0,0,0,0
1,2024-04-14 15:30:00,Arsenal22216,Arsenal,h,1.986,0.929,0,2,18,11,...,22216,1,0,1,1,0,0,1,0,0
2,2024-04-06 16:30:00,Arsenal22206,Arsenal,a,4.165,0.429,3,0,20,10,...,22206,1,0,2,1,1,0,0,0,0
3,2024-04-03 18:30:00,Arsenal22196,Arsenal,h,1.194,0.292,2,0,13,6,...,22196,1,0,3,1,1,0,0,0,0
4,2024-03-31 16:30:00,Arsenal22190,Arsenal,a,0.511,1.220,0,0,6,12,...,22190,1,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2024-03-09 15:00:00,Wolverhampton Wanderers22174,Wolverhampton Wanderers,h,1.296,2.972,2,1,8,24,...,22174,1,0,3,2,1,0,0,1,0
211,2024-03-02 15:00:00,Wolverhampton Wanderers22161,Wolverhampton Wanderers,a,0.511,3.417,0,3,12,14,...,22161,1,0,2,2,0,0,1,0,0
212,2024-02-25 13:30:00,Wolverhampton Wanderers22154,Wolverhampton Wanderers,h,1.216,0.657,1,0,13,12,...,22154,1,0,3,2,1,0,0,0,0
213,2024-02-17 15:00:00,Wolverhampton Wanderers22144,Wolverhampton Wanderers,a,1.790,1.603,2,1,12,15,...,22144,1,0,1,2,1,0,0,1,0


In [7]:
# Home/Away Tables last 10 games (Form) - Relegated Teams are removed from index

eng_r = ['Southampton', 'Leicester','Leeds']
fra_r = ['Angers','Ajaccio','Troyes','Auxerre']
ger_r = ['Schalke 04','Hertha Berlin']
ita_r = ['Sampdoria','Spezia','Cremonese']
spa_r = ['Espanyol','Elche','Real Valladolid']

def home_away_table_form(df,relegated_list):
    # changes to df
    df['match_ID'] = df['ID'].apply(lambda x:x[-5:])
    df['match_ID'] = pd.to_numeric(df['match_ID'])
    df['played'] = 1    
    # new df
    df_h_a = df.groupby(["Team","H/A"]).agg({'played':'sum', 'xG':'sum', 'xGC':'sum','G':'sum', 'GC':'sum', 'Pts':'sum',
                                             'S':'sum', 'SC':'sum', 'QS': 'sum','QSC':'sum',
                                             'BTTS':'sum', 'Blank':'sum', 'CS':'sum','first_goal':'sum','first_conc':'sum',
                                             'btts_if_g':'sum','btts_if_c':'sum','0_0':'sum','Rankvs':'sum'})
    h_a_sep = [x for _, x in df_h_a.groupby("H/A")]
    away_table = h_a_sep[0].droplevel("H/A")
    home_table = h_a_sep[1].droplevel("H/A")
    home_table = home_table.apply(pd.to_numeric)
    away_table = away_table.apply(pd.to_numeric)
    home_table['GD'] = home_table['G'] - home_table['GC']
    away_table['GD'] = away_table['G'] - away_table['GC']
    home_table['sample'] = home_table['played']**2
    away_table['sample'] = home_table['played']**2
    home_table = home_table.iloc[:,1:].div(home_table.played, axis=0)
    away_table = away_table.iloc[:,1:].div(away_table.played, axis=0)
    home_table = home_table[~home_table.index.isin(relegated_list)]
    away_table = away_table[~away_table.index.isin(relegated_list)]


    q1_h = home_table['Pts'].quantile(0.25)
    q1_a = away_table['Pts'].quantile(0.25)
    q3_h = home_table['Pts'].quantile(0.75)
    q3_a = away_table['Pts'].quantile(0.75)
    home_table['Rank'] = home_table.apply(lambda x: 3 if x['Pts'] <= q1_h else( 1 if x['Pts'] >= q3_h else 2), axis=1) 
    away_table['Rank'] = away_table.apply(lambda x: 3 if x['Pts'] <= q1_a else( 1 if x['Pts'] >= q3_a else 2), axis=1) 

    home_table = home_table.sort_values('Pts', ascending=False)
    away_table = away_table.sort_values('Pts', ascending=False)

    return home_table, away_table

eng_l10_table_h,eng_l10_table_a = home_away_table_form(eng_l10,eng_r)
fra_l10_table_h,fra_l10_table_a = home_away_table_form(fra_l10,fra_r)
ger_l10_table_h,ger_l10_table_a = home_away_table_form(ger_l10,ger_r)
ita_l10_table_h,ita_l10_table_a = home_away_table_form(ita_l10,ita_r)
spa_l10_table_h,spa_l10_table_a = home_away_table_form(spa_l10,spa_r)
spa_l10_table_h

,xG,xGC,G,GC,Pts,S,SC,QS,QSC,BTTS,...,CS,first_goal,first_conc,btts_if_g,btts_if_c,0_0,Rankvs,GD,sample,Rank
Team,,,,,,,,,,,,,,,,,,,,,
Real Madrid,2.579500,0.782000,2.500000,0.500000,2.666667,16.000000,9.000000,3.333333,0.833333,0.333333,...,0.666667,0.833333,0.166667,0.166667,0.166667,0.000000,1.666667,2.000000,6.0,1
Girona,2.316000,0.736600,2.400000,0.600000,2.600000,11.000000,8.800000,3.600000,1.000000,0.400000,...,0.600000,0.800000,0.000000,0.400000,0.000000,0.200000,2.000000,1.800000,5.0,1
Barcelona,2.250000,0.585250,2.250000,0.750000,2.500000,15.250000,9.500000,3.250000,0.250000,0.250000,...,0.750000,1.000000,0.000000,0.250000,0.000000,0.000000,2.500000,1.500000,4.0,1
Atletico Madrid,2.615500,1.361500,2.500000,1.250000,2.250000,14.750000,11.250000,3.750000,2.000000,0.500000,...,0.250000,0.500000,0.500000,0.250000,0.250000,0.000000,1.500000,1.250000,4.0,1
Athletic Club,1.667000,1.120667,1.833333,0.666667,2.000000,13.666667,8.000000,2.000000,1.500000,0.500000,...,0.500000,0.666667,0.166667,0.333333,0.166667,0.166667,2.000000,1.166667,6.0,1
Celta Vigo,1.721750,0.815500,1.500000,0.750000,1.750000,15.500000,8.500000,2.250000,0.500000,0.500000,...,0.500000,0.250000,0.500000,0.000000,0.500000,0.250000,2.000000,0.750000,4.0,2
Sevilla,1.940250,1.600750,1.500000,1.250000,1.750000,10.750000,14.500000,3.500000,1.750000,0.750000,...,0.250000,1.000000,0.000000,0.750000,0.000000,0.000000,1.750000,0.250000,4.0,2
Villarreal,1.730600,0.851600,1.600000,0.800000,1.600000,15.800000,11.200000,2.000000,0.600000,0.600000,...,0.400000,0.400000,0.400000,0.200000,0.400000,0.200000,2.400000,0.800000,5.0,2
Real Betis,1.262400,1.127000,1.600000,1.200000,1.600000,12.800000,9.200000,1.600000,1.400000,0.800000,...,0.200000,0.600000,0.200000,0.600000,0.200000,0.200000,2.000000,0.400000,5.0,2


In [8]:
# Rankvs Tables for home/away performance against ranks 1,2 and 3

def home_away_df(df):

    df_h = df[df['H/A'] == "h"]
    df_a = df[df['H/A'] == "a"]
    return df_h,df_a

eng_home,eng_away = home_away_df(eng_df_updated)
fra_home,fra_away = home_away_df(fra_df_updated)
ger_home,ger_away = home_away_df(ger_df_updated)
ita_home,ita_away = home_away_df(ita_df_updated)
spa_home,spa_away = home_away_df(spa_df_updated)


def home_away_rank_df(df_h,df_a):
    df1_h = df_h[df_h['Rankvs'] == 1]
    df2_h = df_h[df_h['Rankvs'] == 2]
    df3_h = df_h[df_h['Rankvs'] == 3]
    
    df1_a = df_a[df_a['Rankvs'] == 1]
    df2_a = df_a[df_a['Rankvs'] == 2]
    df3_a = df_a[df_a['Rankvs'] == 3]
    return df1_h, df2_h, df3_h,df1_a,df2_a,df3_a

eng_r1_h_df, eng_r2_h_df, eng_r3_h_df, eng_r1_a_df, eng_r2_a_df,eng_r3_a_df = home_away_rank_df(eng_home,eng_away)
fra_r1_h_df, fra_r2_h_df, fra_r3_h_df, fra_r1_a_df, fra_r2_a_df,fra_r3_a_df = home_away_rank_df(fra_home,fra_away)
ger_r1_h_df, ger_r2_h_df, ger_r3_h_df, ger_r1_a_df, ger_r2_a_df,ger_r3_a_df = home_away_rank_df(ger_home,ger_away)
ita_r1_h_df, ita_r2_h_df, ita_r3_h_df, ita_r1_a_df, ita_r2_a_df,ita_r3_a_df = home_away_rank_df(ita_home,ita_away)
spa_r1_h_df, spa_r2_h_df, spa_r3_h_df, spa_r1_a_df, spa_r2_a_df,spa_r3_a_df = home_away_rank_df(spa_home,spa_away)
eng_r2_a_df




,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,...,played,0_0,Rankvs,Rank,win,draw,lose,date_int,btts_if_g,btts_if_c
0,2024-04-21 15:30:00,Liverpool22229,Liverpool,a,1.024,0.851,3,1,14,12,...,1,0,2,1,1,0,0,1 days 02:11:26.565033,1,0
0,2024-04-21 14:00:00,West Ham22227,West Ham,a,1.345,3.863,2,5,5,19,...,1,0,2,2,0,0,1,1 days 03:41:26.565033,0,1
0,2024-04-20 18:30:00,Arsenal22234,Arsenal,a,1.662,0.165,2,0,24,5,...,1,0,2,1,1,0,0,1 days 23:11:26.565033,0,0
0,2024-04-15 19:00:00,Everton22219,Everton,a,1.218,3.864,0,6,10,14,...,1,0,2,2,0,0,1,6 days 22:41:26.565033,0,0
0,2024-04-14 13:00:00,Fulham22224,Fulham,a,3.633,1.453,2,0,18,15,...,1,0,2,3,1,0,0,8 days 04:41:26.565033,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-04-30 13:00:00,Aston Villa18540,Aston Villa,a,0.435,1.322,0,1,7,14,...,1,0,2,1,0,0,1,358 days 04:41:26.565033,0,0
0,2023-04-30 13:00:00,Southampton18541,Southampton,a,0.897,4.605,1,3,5,22,...,1,0,2,3,0,0,1,358 days 04:41:26.565033,1,0
0,2023-04-29 14:00:00,Wolverhampton Wanderers18535,Wolverhampton Wanderers,a,0.604,3.691,0,6,10,22,...,1,0,2,2,0,0,1,359 days 03:41:26.565033,0,0
0,2023-04-29 14:00:00,Nottingham Forest18534,Nottingham Forest,a,0.965,1.375,1,2,5,14,...,1,0,2,3,0,0,1,359 days 03:41:26.565033,1,0


In [9]:
# Tables based on dataframes for rankvs against ranks 1,2 and 3

def home_away_rank_table(df):
    # changes to df
    df['match_ID'] = df['ID'].apply(lambda x:x[-5:])
    df['match_ID'] = pd.to_numeric(df['match_ID'])
    df['played'] = 1
    
    # new df
    df_h_a = df.groupby(["Team","H/A","Rankvs"]).agg({'played':'sum', 'xG':'sum', 'xGC':'sum','G':'sum', 'GC':'sum', 'Pts':'sum',
                                             'S':'sum', 'SC':'sum', 'QS': 'sum','QSC':'sum',
                                             'BTTS':'sum', 'Blank':'sum', 'CS':'sum','first_goal':'sum','first_conc':'sum',
                                             '0_0':'sum'})
    h_a_sep = [x for _, x in df_h_a.groupby(["H/A","Rankvs"])]
    r1_h_table = h_a_sep[3].droplevel("H/A")
    r2_h_table = h_a_sep[4].droplevel("H/A")
    r3_h_table = h_a_sep[5].droplevel("H/A")
    r1_a_table = h_a_sep[0].droplevel("H/A")
    r2_a_table = h_a_sep[1].droplevel("H/A")
    r3_a_table = h_a_sep[2].droplevel("H/A")

    r1_h_table = r1_h_table.apply(pd.to_numeric)
    r2_h_table = r2_h_table.apply(pd.to_numeric)
    r3_h_table = r3_h_table.apply(pd.to_numeric)
    r1_a_table = r1_a_table.apply(pd.to_numeric)
    r2_a_table = r2_a_table.apply(pd.to_numeric)
    r3_a_table = r3_a_table.apply(pd.to_numeric)
   
    r1_h_table['GD'] = r1_h_table['G'] - r1_h_table['GC']
    r2_h_table['GD'] = r2_h_table['G'] - r2_h_table['GC']
    r3_h_table['GD'] = r3_h_table['G'] - r3_h_table['GC']
    r1_a_table['GD'] = r1_a_table['G'] - r1_a_table['GC']
    r2_a_table['GD'] = r2_a_table['G'] - r2_a_table['GC']
    r3_a_table['GD'] = r3_a_table['G'] - r3_a_table['GC']

    r1_h_table['sample'] = r1_h_table['played'] ** 2
    r2_h_table['sample'] = r2_h_table['played'] ** 2
    r3_h_table['sample'] = r3_h_table['played'] ** 2
    r1_a_table['sample'] = r1_a_table['played'] ** 2
    r2_a_table['sample'] = r2_a_table['played'] ** 2
    r3_a_table['sample'] = r3_a_table['played'] ** 2

    r1_h_table = r1_h_table.iloc[:,1:].div(r1_h_table.played, axis=0)
    r2_h_table = r2_h_table.iloc[:,1:].div(r2_h_table.played, axis=0)
    r3_h_table = r3_h_table.iloc[:,1:].div(r3_h_table.played, axis=0)
    r1_a_table = r1_a_table.iloc[:,1:].div(r1_a_table.played, axis=0)
    r2_a_table = r2_a_table.iloc[:,1:].div(r2_a_table.played, axis=0)
    r3_a_table = r3_a_table.iloc[:,1:].div(r3_a_table.played, axis=0)

    r1_h_table['btts_if_g'] = r1_h_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    r2_h_table['btts_if_g'] = r2_h_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    r3_h_table['btts_if_g'] = r3_h_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    r1_a_table['btts_if_g'] = r1_a_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    r2_a_table['btts_if_g'] = r2_a_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    r3_a_table['btts_if_g'] = r3_a_table.apply(lambda x: 0 if x['first_goal'] == 0 else ((x['first_goal']-(x['CS']-x['0_0']))/x['first_goal']), axis = 1)
    r1_h_table['btts_if_c'] = r1_h_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)
    r2_h_table['btts_if_c'] = r2_h_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)
    r3_h_table['btts_if_c'] = r3_h_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)
    r1_a_table['btts_if_c'] = r1_a_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)
    r2_a_table['btts_if_c'] = r2_a_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)
    r3_a_table['btts_if_c'] = r3_a_table.apply(lambda x: 0 if x['first_conc'] == 0 else ((x['first_conc']-(x['Blank']-x['0_0']))/x['first_conc']), axis = 1)

    r1_h_table['home?'] = 1
    r2_h_table['home?'] = 1
    r3_h_table['home?'] = 1
    r1_a_table['home?'] = 0
    r2_a_table['home?'] = 0
    r3_a_table['home?'] = 0

    r1_h_table['GD'] = r1_h_table['G'] - r1_h_table['GC']
    r2_h_table['GD'] = r2_h_table['G'] - r2_h_table['GC']
    r3_h_table['GD'] = r3_h_table['G'] - r3_h_table['GC']
    r1_a_table['GD'] = r1_a_table['G'] - r1_a_table['GC']
    r2_a_table['GD'] = r2_a_table['G'] - r2_a_table['GC']
    r3_a_table['GD'] = r3_a_table['G'] - r3_a_table['GC']

    r1_h_table = r1_h_table.sort_values('Pts', ascending=False)
    r2_h_table = r2_h_table.sort_values('Pts', ascending=False)
    r3_h_table = r3_h_table.sort_values('Pts', ascending=False)
    r1_a_table = r1_a_table.sort_values('Pts', ascending=False)
    r2_a_table = r2_a_table.sort_values('Pts', ascending=False)
    r3_a_table = r3_a_table.sort_values('Pts', ascending=False)

    return r1_h_table, r2_h_table, r3_h_table, r1_a_table, r2_a_table, r3_a_table

r1_h_eng,r2_h_eng,r3_h_eng,r1_a_eng,r2_a_eng,r3_a_eng = home_away_rank_table(eng_df_updated)
r1_h_fra,r2_h_fra,r3_h_fra,r1_a_fra,r2_a_fra,r3_a_fra = home_away_rank_table(fra_df_updated)
r1_h_ger,r2_h_ger,r3_h_ger,r1_a_ger,r2_a_ger,r3_a_ger = home_away_rank_table(ger_df_updated)
r1_h_ita,r2_h_ita,r3_h_ita,r1_a_ita,r2_a_ita,r3_a_ita = home_away_rank_table(ita_df_updated)
r1_h_spa,r2_h_spa,r3_h_spa,r1_a_spa,r2_a_spa,r3_a_spa = home_away_rank_table(spa_df_updated)
r3_h_eng

,,xG,xGC,G,GC,Pts,S,SC,QS,QSC,BTTS,Blank,CS,first_goal,first_conc,0_0,GD,sample,btts_if_g,btts_if_c,home?
Team,Rankvs,,,,,,,,,,,,,,,,,,,,
Liverpool,3,2.641714,0.945429,3.000000,0.857143,3.000000,23.285714,9.428571,3.285714,1.285714,0.571429,0.000000,0.428571,0.857143,0.142857,0.000000,2.142857,7.0,0.500000,1.000000,1
Brentford,3,2.204000,0.789000,2.600000,0.800000,3.000000,16.800000,7.800000,2.600000,1.200000,0.600000,0.000000,0.400000,0.600000,0.400000,0.000000,1.800000,5.0,0.333333,1.000000,1
Manchester City,3,2.512857,0.832286,2.857143,0.571429,3.000000,18.000000,6.142857,2.857143,1.285714,0.571429,0.000000,0.428571,1.000000,0.000000,0.000000,2.285714,7.0,0.571429,0.000000,1
Arsenal,3,2.089667,0.533167,2.666667,0.833333,2.666667,15.500000,6.500000,2.000000,0.500000,0.666667,0.000000,0.333333,0.833333,0.166667,0.000000,1.833333,6.0,0.600000,1.000000,1
Tottenham,3,2.182833,1.133333,2.166667,1.333333,2.500000,19.500000,10.000000,2.500000,1.666667,0.833333,0.000000,0.166667,0.500000,0.500000,0.000000,0.833333,6.0,0.666667,1.000000,1
Manchester United,3,2.364400,1.083800,1.800000,1.200000,2.400000,19.200000,11.400000,3.400000,1.000000,0.800000,0.000000,0.200000,0.200000,0.800000,0.000000,0.600000,5.0,0.000000,1.000000,1
Aston Villa,3,2.561167,1.095667,2.833333,1.666667,2.333333,14.500000,8.333333,3.000000,1.500000,1.000000,0.000000,0.000000,0.833333,0.166667,0.000000,1.166667,6.0,1.000000,1.000000,1
Bournemouth,3,2.165200,0.999600,2.400000,1.000000,2.200000,16.400000,10.000000,2.800000,1.000000,0.800000,0.000000,0.200000,0.600000,0.400000,0.000000,1.400000,5.0,0.666667,1.000000,1
West Ham,3,2.235333,1.487000,2.333333,1.500000,2.166667,18.333333,13.500000,3.166667,1.666667,0.666667,0.166667,0.166667,0.500000,0.500000,0.000000,0.833333,6.0,0.666667,0.666667,1


In [10]:
# ML DFs for Rank vs Rank training

def rank_vs_rank_df(df1h,df2h,df3h,df1a,df2a,df3a):
    r1v1_h = df1h[df1h['Rank'] == 1]
    r2v1_h = df1h[df1h['Rank'] == 2]
    r3v1_h = df1h[df1h['Rank'] == 3]
    r1v2_h = df2h[df2h['Rank'] == 1]
    r2v2_h = df2h[df2h['Rank'] == 2]
    r3v2_h = df2h[df2h['Rank'] == 3]
    r1v3_h = df3h[df3h['Rank'] == 1]
    r2v3_h = df3h[df3h['Rank'] == 2]
    r3v3_h = df3h[df3h['Rank'] == 3]

    r1v1_a = df1a[df1a['Rank'] == 1]
    r2v1_a = df1a[df1a['Rank'] == 2]
    r3v1_a = df1a[df1a['Rank'] == 3]
    r1v2_a = df2a[df2a['Rank'] == 1]
    r2v2_a = df2a[df2a['Rank'] == 2]
    r3v2_a = df2a[df2a['Rank'] == 3]
    r1v3_a = df3a[df3a['Rank'] == 1]
    r2v3_a = df3a[df3a['Rank'] == 2]
    r3v3_a = df3a[df3a['Rank'] == 3]

    return r1v1_h, r2v1_h, r3v1_h, r1v2_h, r2v2_h, r3v2_h, r1v3_h, r2v3_h, r3v3_h, r1v1_a, r2v1_a, r3v1_a, r1v2_a, r2v2_a, r3v2_a, r1v3_a, r2v3_a, r3v3_a

eng_1v1_df_h, eng_2v1_df_h, eng_3v1_df_h, eng_1v2_df_h, eng_2v2_df_h, eng_3v2_df_h, eng_1v3_df_h, eng_2v3_df_h, eng_3v3_df_h, eng_1v1_df_a, eng_2v1_df_a, eng_3v1_df_a, eng_1v2_df_a, eng_2v2_df_a, eng_3v2_df_a, eng_1v3_df_a, eng_2v3_df_a, eng_3v3_df_a = rank_vs_rank_df(eng_r1_h_df, eng_r2_h_df,eng_r3_h_df,eng_r1_a_df,eng_r2_a_df,eng_r3_a_df) 
fra_1v1_df_h, fra_2v1_df_h, fra_3v1_df_h, fra_1v2_df_h, fra_2v2_df_h, fra_3v2_df_h, fra_1v3_df_h, fra_2v3_df_h, fra_3v3_df_h, fra_1v1_df_a, fra_2v1_df_a, fra_3v1_df_a, fra_1v2_df_a, fra_2v2_df_a, fra_3v2_df_a, fra_1v3_df_a, fra_2v3_df_a, fra_3v3_df_a = rank_vs_rank_df(fra_r1_h_df, fra_r2_h_df,fra_r3_h_df,fra_r1_a_df,fra_r2_a_df,fra_r3_a_df) 
ger_1v1_df_h, ger_2v1_df_h, ger_3v1_df_h, ger_1v2_df_h, ger_2v2_df_h, ger_3v2_df_h, ger_1v3_df_h, ger_2v3_df_h, ger_3v3_df_h, ger_1v1_df_a, ger_2v1_df_a, ger_3v1_df_a, ger_1v2_df_a, ger_2v2_df_a, ger_3v2_df_a, ger_1v3_df_a, ger_2v3_df_a, ger_3v3_df_a = rank_vs_rank_df(ger_r1_h_df, ger_r2_h_df,ger_r3_h_df,ger_r1_a_df,ger_r2_a_df,ger_r3_a_df) 
ita_1v1_df_h, ita_2v1_df_h, ita_3v1_df_h, ita_1v2_df_h, ita_2v2_df_h, ita_3v2_df_h, ita_1v3_df_h, ita_2v3_df_h, ita_3v3_df_h, ita_1v1_df_a, ita_2v1_df_a, ita_3v1_df_a, ita_1v2_df_a, ita_2v2_df_a, ita_3v2_df_a, ita_1v3_df_a, ita_2v3_df_a, ita_3v3_df_a = rank_vs_rank_df(ita_r1_h_df, ita_r2_h_df,ita_r3_h_df,ita_r1_a_df,ita_r2_a_df,ita_r3_a_df) 
spa_1v1_df_h, spa_2v1_df_h, spa_3v1_df_h, spa_1v2_df_h, spa_2v2_df_h, spa_3v2_df_h, spa_1v3_df_h, spa_2v3_df_h, spa_3v3_df_h, spa_1v1_df_a, spa_2v1_df_a, spa_3v1_df_a, spa_1v2_df_a, spa_2v2_df_a, spa_3v2_df_a, spa_1v3_df_a, spa_2v3_df_a, spa_3v3_df_a = rank_vs_rank_df(spa_r1_h_df, spa_r2_h_df,spa_r3_h_df,spa_r1_a_df,spa_r2_a_df,spa_r3_a_df) 
spa_3v3_df_a




,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,...,played,0_0,Rankvs,Rank,win,draw,lose,date_int,btts_if_g,btts_if_c
0,2024-04-14 14:15:00,Alaves22994,Alaves,a,0.279,1.893,0,2,8,9,...,1,0,3,3,0,0,1,8 days 03:26:26.802986,0,0
0,2024-03-02 15:15:00,Cadiz22949,Cadiz,a,1.216,0.688,1,1,15,9,...,1,0,3,3,0,1,0,51 days 02:26:26.802986,0,1
0,2024-03-01 20:00:00,Almeria22948,Almeria,a,0.330,1.823,0,1,3,15,...,1,0,3,3,0,0,1,51 days 21:41:26.802986,0,0
0,2024-02-18 15:15:00,Almeria22929,Almeria,a,0.699,1.796,1,1,13,19,...,1,0,3,3,0,1,0,64 days 02:26:26.802986,1,0
0,2024-01-26 20:00:00,Alaves22904,Alaves,a,2.061,1.808,3,0,11,19,...,1,0,3,3,1,0,0,86 days 21:41:26.802986,0,0
0,2024-01-12 20:00:00,Alaves22884,Alaves,a,2.141,2.497,3,2,16,19,...,1,0,3,3,1,0,0,100 days 21:41:26.802986,1,0
0,2024-01-03 16:00:00,Cadiz22872,Cadiz,a,0.747,3.877,0,2,15,15,...,1,0,3,3,0,0,1,110 days 01:41:26.802986,0,0
0,2023-12-17 13:00:00,Mallorca22845,Mallorca,a,1.514,0.315,0,0,20,7,...,1,1,3,3,0,1,0,127 days 04:41:26.802986,0,0
0,2023-12-16 13:00:00,Granada22847,Granada,a,0.690,1.351,0,1,7,11,...,1,0,3,3,0,0,1,128 days 04:41:26.802986,0,0
0,2023-12-04 20:00:00,Cadiz22831,Cadiz,a,0.347,2.736,1,1,2,26,...,1,0,3,3,0,1,0,139 days 21:41:26.802986,1,0


In [11]:
# Posting relevant tables and dataframes to excel documents per league

# with pd.ExcelWriter("../EDA_DF/EPL_understat.xlsx") as writer: # VSCode local use
with pd.ExcelWriter("EDA_DF/EPL_understat.xlsx") as writer: #GitHub Actions use externally
    eng_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_eng.to_excel(writer, sheet_name="rank1_home")
    r2_h_eng.to_excel(writer, sheet_name="rank2_home")
    r3_h_eng.to_excel(writer, sheet_name="rank3_home")
    r1_a_eng.to_excel(writer, sheet_name="rank1_away")
    r2_a_eng.to_excel(writer, sheet_name="rank2_away")
    r3_a_eng.to_excel(writer, sheet_name="rank3_away")
    eng_r1_h_df.to_excel(writer, sheet_name="versus_rank1_home_df")
    eng_r2_h_df.to_excel(writer, sheet_name="versus_rank2_home_df")
    eng_r3_h_df.to_excel(writer, sheet_name="versus_rank3_home_df")
    eng_r1_a_df.to_excel(writer, sheet_name="versus_rank1_away_df")
    eng_r2_a_df.to_excel(writer, sheet_name="versus_rank2_away_df")
    eng_r3_a_df.to_excel(writer, sheet_name="versus_rank3_away_df")
    eng_l10.to_excel(writer, sheet_name='last10_gamesheet')
    eng_l10_table_h.to_excel(writer, sheet_name="last10_home")
    eng_l10_table_a.to_excel(writer, sheet_name="last10_away")
    eng_table_h.to_excel(writer, sheet_name="home")
    eng_table_a.to_excel(writer, sheet_name="away")

# with pd.ExcelWriter("../EDA_DF/Ligue1_understat.xlsx") as writer: # VSCode local use
with pd.ExcelWriter("EDA_DF/Ligue1_understat.xlsx") as writer: # GitHub Actions use externally
    fra_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_fra.to_excel(writer, sheet_name="rank1_home")
    r2_h_fra.to_excel(writer, sheet_name="rank2_home")
    r3_h_fra.to_excel(writer, sheet_name="rank3_home")
    r1_a_fra.to_excel(writer, sheet_name="rank1_away")
    r2_a_fra.to_excel(writer, sheet_name="rank2_away")
    r3_a_fra.to_excel(writer, sheet_name="rank3_away")
    fra_r1_h_df.to_excel(writer, sheet_name="versus_rank1_home_df")
    fra_r2_h_df.to_excel(writer, sheet_name="versus_rank2_home_df")
    fra_r3_h_df.to_excel(writer, sheet_name="versus_rank3_home_df")
    fra_r1_a_df.to_excel(writer, sheet_name="versus_rank1_away_df")
    fra_r2_a_df.to_excel(writer, sheet_name="versus_rank2_away_df")
    fra_r3_a_df.to_excel(writer, sheet_name="versus_rank3_away_df")
    fra_l10.to_excel(writer, sheet_name='last10_gamesheet')
    fra_l10_table_h.to_excel(writer, sheet_name="last10_home")
    fra_l10_table_a.to_excel(writer, sheet_name="last10_away")
    fra_table_h.to_excel(writer, sheet_name="home")
    fra_table_a.to_excel(writer, sheet_name="away")

# with pd.ExcelWriter("../EDA_DF/Bundesliga_understat.xlsx") as writer: # VSCode local use
with pd.ExcelWriter("EDA_DF/Bundesliga_understat.xlsx") as writer: # GitHub Actions use externally
    ger_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_ger.to_excel(writer, sheet_name="rank1_home")
    r2_h_ger.to_excel(writer, sheet_name="rank2_home")
    r3_h_ger.to_excel(writer, sheet_name="rank3_home")
    r1_a_ger.to_excel(writer, sheet_name="rank1_away")
    r2_a_ger.to_excel(writer, sheet_name="rank2_away")
    r3_a_ger.to_excel(writer, sheet_name="rank3_away")
    ger_r1_h_df.to_excel(writer, sheet_name="versus_rank1_home_df")
    ger_r2_h_df.to_excel(writer, sheet_name="versus_rank2_home_df")
    ger_r3_h_df.to_excel(writer, sheet_name="versus_rank3_home_df")
    ger_r1_a_df.to_excel(writer, sheet_name="versus_rank1_away_df")
    ger_r2_a_df.to_excel(writer, sheet_name="versus_rank2_away_df")
    ger_r3_a_df.to_excel(writer, sheet_name="versus_rank3_away_df")
    ger_l10.to_excel(writer, sheet_name='last10_gamesheet')
    ger_l10_table_h.to_excel(writer, sheet_name="last10_home")
    ger_l10_table_a.to_excel(writer, sheet_name="last10_away")
    ger_table_h.to_excel(writer, sheet_name="home")
    ger_table_a.to_excel(writer, sheet_name="away")

# with pd.ExcelWriter("../EDA_DF/SerieA_understat.xlsx") as writer: # VSCode local use
with pd.ExcelWriter("EDA_DF/SerieA_understat.xlsx") as writer: # GitHub Actions use externally
    ita_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_ita.to_excel(writer, sheet_name="rank1_home")
    r2_h_ita.to_excel(writer, sheet_name="rank2_home")
    r3_h_ita.to_excel(writer, sheet_name="rank3_home")
    r1_a_ita.to_excel(writer, sheet_name="rank1_away")
    r2_a_ita.to_excel(writer, sheet_name="rank2_away")
    r3_a_ita.to_excel(writer, sheet_name="rank3_away")
    ita_r1_h_df.to_excel(writer, sheet_name="versus_rank1_home_df")
    ita_r2_h_df.to_excel(writer, sheet_name="versus_rank2_home_df")
    ita_r3_h_df.to_excel(writer, sheet_name="versus_rank3_home_df")
    ita_r1_a_df.to_excel(writer, sheet_name="versus_rank1_away_df")
    ita_r2_a_df.to_excel(writer, sheet_name="versus_rank2_away_df")
    ita_r3_a_df.to_excel(writer, sheet_name="versus_rank3_away_df")
    ita_l10.to_excel(writer, sheet_name='last10_gamesheet')
    ita_l10_table_h.to_excel(writer, sheet_name="last10_home")
    ita_l10_table_a.to_excel(writer, sheet_name="last10_away")
    ita_table_h.to_excel(writer, sheet_name="home")
    ita_table_a.to_excel(writer, sheet_name="away")
    
# with pd.ExcelWriter("../EDA_DF/LaLiga_understat.xlsx") as writer: # VSCode local use
with pd.ExcelWriter("EDA_DF/LaLiga_understat.xlsx") as writer: # GitHub Actions use externally
    spa_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_spa.to_excel(writer, sheet_name="rank1_home")
    r2_h_spa.to_excel(writer, sheet_name="rank2_home")
    r3_h_spa.to_excel(writer, sheet_name="rank3_home")
    r1_a_spa.to_excel(writer, sheet_name="rank1_away")
    r2_a_spa.to_excel(writer, sheet_name="rank2_away")
    r3_a_spa.to_excel(writer, sheet_name="rank3_away")
    spa_r1_h_df.to_excel(writer, sheet_name="versus_rank1_home_df")
    spa_r2_h_df.to_excel(writer, sheet_name="versus_rank2_home_df")
    spa_r3_h_df.to_excel(writer, sheet_name="versus_rank3_home_df")
    spa_r1_a_df.to_excel(writer, sheet_name="versus_rank1_away_df")
    spa_r2_a_df.to_excel(writer, sheet_name="versus_rank2_away_df")
    spa_r3_a_df.to_excel(writer, sheet_name="versus_rank3_away_df")
    spa_l10.to_excel(writer, sheet_name='last10_gamesheet')
    spa_l10_table_h.to_excel(writer, sheet_name="last10_home")
    spa_l10_table_a.to_excel(writer, sheet_name="last10_away")
    spa_table_h.to_excel(writer, sheet_name="home")
    spa_table_a.to_excel(writer, sheet_name="away")

In [35]:
# For ML learning model

# with pd.ExcelWriter("EDA_DF/EPL_Rank_ML.xlsx") as writer:
#     eng_1v1_df_h.to_excel(writer, sheet_name="1v1_home")
#     eng_2v1_df_h.to_excel(writer, sheet_name="2v1_home")
#     eng_3v1_df_h.to_excel(writer, sheet_name="3v1_home")
#     eng_1v2_df_h.to_excel(writer, sheet_name="1v2_home")
#     eng_2v2_df_h.to_excel(writer, sheet_name="2v2_home")
#     eng_3v2_df_h.to_excel(writer, sheet_name="3v2_home")
#     eng_1v3_df_h.to_excel(writer, sheet_name="1v3_home")
#     eng_2v3_df_h.to_excel(writer, sheet_name="2v3_home")
#     eng_3v3_df_h.to_excel(writer, sheet_name="3v3_home")
#     eng_1v1_df_a.to_excel(writer, sheet_name="1v1_away")
#     eng_2v1_df_a.to_excel(writer, sheet_name="2v1_away")
#     eng_3v1_df_a.to_excel(writer, sheet_name="3v1_away")
#     eng_1v2_df_a.to_excel(writer, sheet_name="1v2_away")
#     eng_2v2_df_a.to_excel(writer, sheet_name="2v2_away")
#     eng_3v2_df_a.to_excel(writer, sheet_name="3v2_away")
#     eng_1v3_df_a.to_excel(writer, sheet_name="1v3_away")
#     eng_2v3_df_a.to_excel(writer, sheet_name="2v3_away")
#     eng_3v3_df_a.to_excel(writer, sheet_name="3v3_away")

# with pd.ExcelWriter("EDA_DF/Ligue1_Rank_ML.xlsx") as writer:
#     fra_1v1_df_h.to_excel(writer, sheet_name="1v1_home")
#     fra_2v1_df_h.to_excel(writer, sheet_name="2v1_home")
#     fra_3v1_df_h.to_excel(writer, sheet_name="3v1_home")
#     fra_1v2_df_h.to_excel(writer, sheet_name="1v2_home")
#     fra_2v2_df_h.to_excel(writer, sheet_name="2v2_home")
#     fra_3v2_df_h.to_excel(writer, sheet_name="3v2_home")
#     fra_1v3_df_h.to_excel(writer, sheet_name="1v3_home")
#     fra_2v3_df_h.to_excel(writer, sheet_name="2v3_home")
#     fra_3v3_df_h.to_excel(writer, sheet_name="3v3_home")
#     fra_1v1_df_a.to_excel(writer, sheet_name="1v1_away")
#     fra_2v1_df_a.to_excel(writer, sheet_name="2v1_away")
#     fra_3v1_df_a.to_excel(writer, sheet_name="3v1_away")
#     fra_1v2_df_a.to_excel(writer, sheet_name="1v2_away")
#     fra_2v2_df_a.to_excel(writer, sheet_name="2v2_away")
#     fra_3v2_df_a.to_excel(writer, sheet_name="3v2_away")
#     fra_1v3_df_a.to_excel(writer, sheet_name="1v3_away")
#     fra_2v3_df_a.to_excel(writer, sheet_name="2v3_away")
#     fra_3v3_df_a.to_excel(writer, sheet_name="3v3_away")

# with pd.ExcelWriter("EDA_DF/Bundesliga_Rank_ML.xlsx") as writer:
#     ger_1v1_df_h.to_excel(writer, sheet_name="1v1_home")
#     ger_2v1_df_h.to_excel(writer, sheet_name="2v1_home")
#     ger_3v1_df_h.to_excel(writer, sheet_name="3v1_home")
#     ger_1v2_df_h.to_excel(writer, sheet_name="1v2_home")
#     ger_2v2_df_h.to_excel(writer, sheet_name="2v2_home")
#     ger_3v2_df_h.to_excel(writer, sheet_name="3v2_home")
#     ger_1v3_df_h.to_excel(writer, sheet_name="1v3_home")
#     ger_2v3_df_h.to_excel(writer, sheet_name="2v3_home")
#     ger_3v3_df_h.to_excel(writer, sheet_name="3v3_home")
#     ger_1v1_df_a.to_excel(writer, sheet_name="1v1_away")
#     ger_2v1_df_a.to_excel(writer, sheet_name="2v1_away")
#     ger_3v1_df_a.to_excel(writer, sheet_name="3v1_away")
#     ger_1v2_df_a.to_excel(writer, sheet_name="1v2_away")
#     ger_2v2_df_a.to_excel(writer, sheet_name="2v2_away")
#     ger_3v2_df_a.to_excel(writer, sheet_name="3v2_away")
#     ger_1v3_df_a.to_excel(writer, sheet_name="1v3_away")
#     ger_2v3_df_a.to_excel(writer, sheet_name="2v3_away")
#     ger_3v3_df_a.to_excel(writer, sheet_name="3v3_away")

# with pd.ExcelWriter("EDA_DF/SerieA_Rank_ML.xlsx") as writer:
#     ita_1v1_df_h.to_excel(writer, sheet_name="1v1_home")
#     ita_2v1_df_h.to_excel(writer, sheet_name="2v1_home")
#     ita_3v1_df_h.to_excel(writer, sheet_name="3v1_home")
#     ita_1v2_df_h.to_excel(writer, sheet_name="1v2_home")
#     ita_2v2_df_h.to_excel(writer, sheet_name="2v2_home")
#     ita_3v2_df_h.to_excel(writer, sheet_name="3v2_home")
#     ita_1v3_df_h.to_excel(writer, sheet_name="1v3_home")
#     ita_2v3_df_h.to_excel(writer, sheet_name="2v3_home")
#     ita_3v3_df_h.to_excel(writer, sheet_name="3v3_home")
#     ita_1v1_df_a.to_excel(writer, sheet_name="1v1_away")
#     ita_2v1_df_a.to_excel(writer, sheet_name="2v1_away")
#     ita_3v1_df_a.to_excel(writer, sheet_name="3v1_away")
#     ita_1v2_df_a.to_excel(writer, sheet_name="1v2_away")
#     ita_2v2_df_a.to_excel(writer, sheet_name="2v2_away")
#     ita_3v2_df_a.to_excel(writer, sheet_name="3v2_away")
#     ita_1v3_df_a.to_excel(writer, sheet_name="1v3_away")
#     ita_2v3_df_a.to_excel(writer, sheet_name="2v3_away")
#     ita_3v3_df_a.to_excel(writer, sheet_name="3v3_away")

# with pd.ExcelWriter("EDA_DF/LaLiga_Rank_ML.xlsx") as writer:
#     spa_1v1_df_h.to_excel(writer, sheet_name="1v1_home")
#     spa_2v1_df_h.to_excel(writer, sheet_name="2v1_home")
#     spa_3v1_df_h.to_excel(writer, sheet_name="3v1_home")
#     spa_1v2_df_h.to_excel(writer, sheet_name="1v2_home")
#     spa_2v2_df_h.to_excel(writer, sheet_name="2v2_home")
#     spa_3v2_df_h.to_excel(writer, sheet_name="3v2_home")
#     spa_1v3_df_h.to_excel(writer, sheet_name="1v3_home")
#     spa_2v3_df_h.to_excel(writer, sheet_name="2v3_home")
#     spa_3v3_df_h.to_excel(writer, sheet_name="3v3_home")
#     spa_1v1_df_a.to_excel(writer, sheet_name="1v1_away")
#     spa_2v1_df_a.to_excel(writer, sheet_name="2v1_away")
#     spa_3v1_df_a.to_excel(writer, sheet_name="3v1_away")
#     spa_1v2_df_a.to_excel(writer, sheet_name="1v2_away")
#     spa_2v2_df_a.to_excel(writer, sheet_name="2v2_away")
#     spa_3v2_df_a.to_excel(writer, sheet_name="3v2_away")
#     spa_1v3_df_a.to_excel(writer, sheet_name="1v3_away")
#     spa_2v3_df_a.to_excel(writer, sheet_name="2v3_away")
#     spa_3v3_df_a.to_excel(writer, sheet_name="3v3_away")